## Data Story

In this notebook, I will explore a subset of the data using the Los Angeles files and try to uncover interesting trends and qualities of the Airbnb data.

In [1]:
# import relevant packages
import pandas as pd

In [5]:
# show all columns
pd.set_option('display.max_columns', None)

In [2]:
# load the data
listings = pd.read_csv('/Users/limesncoconuts2/springboard_data/data_capstone_one/csv/los-angeles_listings.csv')
reviews = pd.read_csv('/Users/limesncoconuts2/springboard_data/data_capstone_one/csv/los-angeles_reviews.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,2,4,5,7,9,10,14,16,18,21,22,24,25,28,29,30,31,35,37,40,41,44,45,46,47,48,49,54,59,60,61,67,72,73,74,86,92,93,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
listings.head()

,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,...,space,square_feet,state,street,summary,thumbnail_url,transit,weekly_price_USD,xl_picture_url,zipcode
0,NaN,2.0,"{Internet,""Wireless Internet"",""Air Conditionin...",30.0,365,60,90.0,1,Real Bed,1,...,安靜舒適的独立大套房. 阳光明媚的大卧室，(附帶冰箱)一个超大衣帽間和浴室.,NaN,CA,"South 2nd Avenue, Arcadia, CA 91006, United St...","我們j位於Arcadia市中心,非常适合专业商務人士，学生，渡假遊學...鄰近高速公路,餐厅...",https://a0.muscache.com/ac/pictures/52102815/8...,聚集高尔夫球场，网球场，大型购物中心，影院，图书馆，大型街心公园，跑马场，医院，诊所。非常便...,650.0,https://a0.muscache.com/ac/pictures/52102815/8...,91006.0
1,NaN,1.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30.0,365,60,90.0,1,Real Bed,1,...,NaN,NaN,CA,"North Sycamore Avenue, Los Angeles, CA 90036, ...",Small but real clean apartment in Hancock Park...,https://a2.muscache.com/ac/pictures/23837943/9...,NaN,500.0,https://a2.muscache.com/ac/pictures/23837943/9...,90036.0
2,NaN,6.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0.0,24,0,0.0,3,Real Bed,3,...,NaN,NaN,CA,"Foothill Drive, Los Angeles, CA 90068, United ...","Originally built in 1922, this Italian-style v...",https://a1.muscache.com/ac/pictures/85102851/4...,NaN,NaN,https://a1.muscache.com/ac/pictures/85102851/4...,90068.0
3,NaN,1.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",15.0,344,39,69.0,1,Couch,1,...,Check in is 4pm. Check out is 11am. No excepti...,NaN,CA,"Club Dr, Los Angeles, CA 90064, United States",Extra comfy beanbag that can easily fit two.,https://a0.muscache.com/ac/pictures/74769434/8...,#12 Bus is 100 yards from the apartment and ru...,NaN,https://a0.muscache.com/ac/pictures/74769434/8...,90064.0
4,NaN,2.0,"{""Cable TV"",Internet,""Wireless Internet"",Kitch...",18.0,261,32,62.0,1,Real Bed,1,...,If you are planning to visit our wonderful cit...,175.0,CA,"Martelo Ave, Pasadena, CA 91107, United States",NaN,https://a2.muscache.com/ac/pictures/7556540/fe...,NaN,350.0,https://a2.muscache.com/ac/pictures/7556540/fe...,91107.0


In [7]:
list(listings.columns)

['access',
 'accommodates',
 'amenities',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bed_type',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'calendar_last_scraped',
 'calendar_updated',
 'cancellation_policy',
 'city',
 'cleaning_fee_USD',
 'country',
 'country_code',
 'date_recorded',
 'description',
 'experiences_offered',
 'extra_people_USD',
 'first_review',
 'guests_included',
 'has_availability',
 'host_about',
 'host_acceptance_rate_percentage',
 'host_has_profile_pic',
 'host_id',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_location',
 'host_name',
 'host_neighbourhood',
 'host_picture_url',
 'host_response_rate_percentage',
 'host_response_time',
 'host_since',
 'host_thumbnail_url',
 'host_total_listings_count',
 'host_url',
 'host_verifications',
 'house_rules',
 'id',
 'instant_bookable',
 'interaction',
 'is_business_travel_ready',
 'is_location_exact',
 'jurisdiction_na

In [4]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,date_recorded
0,7680552,50209733,2015-10-10,43618076,Paul,Was a great stay! Host was very helpful.,2017-03-02
1,7680552,50521911,2015-10-12,44347928,Dwendalyz,It was a beautiful home. The kitchen view was ...,2017-03-02
2,7680552,53635576,2015-11-10,31371772,Varunn,The house is a very beautiful house with a gre...,2017-03-02
3,7680552,54673483,2015-11-22,2530719,Lina,Stunning house ! Very clean and comfortable i...,2017-03-02
4,7680552,57881891,2015-12-30,2357565,Sage,Servando was easy to get ahold of as well as p...,2017-03-02


#### Can you count something interesting?

#### Can you find trends (e.g. high, low, increasing, decreasing, anomalies)?

#### Can you make a bar plot or a histogram?

#### Can you compare two related quantities?

#### Can you make a scatterplot?

#### Can you make a time-series plot?

Looking at the plots, what are some insights you can make? Do you see any correlations? Is there a hypothesis you’d like to investigate further? What other questions do the insights lead you to ask?

Now that you’ve asked questions, hopefully you’ve found some interesting insights. Is there a narrative or a way of presenting the insights using text and plots that tells a compelling story? What are some other trends/relationships you think will make the story more complete?